Data processing file to generate the final training and test datasets for classification and prompting.

In [ ]:
!pip install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import sklearn
from sklearn import metrics
import json
import time

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
flute_dataset = load_dataset("ColumbiaNLP/FLUTE",split = 'train')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/ColumbiaNLP___json/ColumbiaNLP--FLUTE-a1b4e1759d13a757/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


In [ ]:
flute_df= pd.DataFrame.from_dict(flute_dataset)

In [ ]:
flute_df[flute_df.apply(lambda x: x['label']=='Entailment' and x['type']=='Metaphor',axis=1)]

,id,hypothesis,premise,idiom,label,explanation,split,type
5,6,My soul was a lampless sea and she was the tem...,My soul was a complete mess and she was the ca...,None,Entailment,"Here, the protagonist's soul was a complete me...",train,Metaphor
10,11,The heavy rain guttered the soil.,The heavy rain made channels in the soil.,None,Entailment,Guttering the soil means that the rain created...,train,Metaphor
11,12,he was feeling down hurt and sad.,he was feeling very depressed.,None,Entailment,Feeling down suggests he was depressed.,train,Metaphor
19,20,The republicans are floating the idea of a tax...,The republicans are proposing the idea of a ta...,None,Entailment,Floating the idea of a tax reform means that t...,train,Metaphor
23,24,The news leaked out despite his secrecy.,the news was revealed publicly despite his sec...,None,Entailment,Leaked out means to reveal the information pub...,train,Metaphor
...,...,...,...,...,...,...,...,...
7494,7495,"Till it drew itself together ,",Till it pulled itself together.,None,Entailment,To draw oneself together means to make an effo...,train,Metaphor
7498,7499,Consumed by the thoughts that grew in the head.,Consumed by the thoughts that were in the head,None,Entailment,The thoughts grew in the head means that the p...,train,Metaphor
7510,7511,He was gripped with a terrible illness that ki...,He was stricken with a terrible illness that k...,None,Entailment,Gripped with illness is used here to mean that...,train,Metaphor
7511,7512,A star crowns the Christmas Tree.,The Star tops the Christmas tree during Christ...,None,Entailment,Crowns the Christmas Tree means it is sitting ...,train,Metaphor


In [ ]:
flute_df['type'].value_counts()

Sarcasm               2216
Idiom                 1768
Simile                1250
Metaphor              1250
CreativeParaphrase    1050
Name: type, dtype: int64

In [ ]:
def update_sarc_premise(row,grp_by_premise):
  premise = row['premise']
  if row['type'] == 'Sarcasm':
    premise_df = grp_by_premise.get_group(premise)
    premise_new = premise_df[premise_df['type'] == 'CreativeParaphrase']['premise'].values
    premise_new = premise_new[0] if len(premise_new)>0 else 'NO PREMISE FOUND'
    return premise_new
  return premise

In [ ]:
from collections import Counter


In [ ]:
def label_similar_sarcasm(row,multiple_sarcasm_premise):
  premise = row['premise']
  type = row['type']
  if premise in multiple_sarcasm_premise and type == 'Sarcasm':
    return 'SIM'
  else:
    return 'UNQ'

In [ ]:
def label_similar_sarc_hypo(row,multiple_sarcasm_premise):
  premise = row['premise']
  type = row['type']
  if premise in multiple_sarcasm_premise and type == 'Sarcasm':
    return 'SIM'
  else:
    return 'UNQ'

In [ ]:
flute_df_mod = flute_df
grp_by_premise = flute_df[flute_df.apply(lambda x : x['type']=='Sarcasm' or x['type'] == 'CreativeParaphrase',axis=1)].groupby('premise')
grp_by_premise.groups
flute_df['premise'] = flute_df.apply(lambda x : update_sarc_premise(x,grp_by_premise),axis = 1)
cnt_dict = {'sarcasm':{4:0,3:0,2:0,1:0,0:0},'creativeparaphrase':{2:0,1:0,0:0}}
multiple_sarcasm_premise = []
for premise in grp_by_premise.groups:
  premise_df = grp_by_premise.get_group(premise)
  if 'Sarcasm' in premise_df['type'].value_counts().keys():
    if premise_df['type'].value_counts()['Sarcasm'] > 1:
      multiple_sarcasm_premise.append(premise)
    cnt_dict['sarcasm'][premise_df['type'].value_counts()['Sarcasm']] += 1
  if 'CreativeParaphrase' in premise_df['type'].value_counts().keys():
    cnt_dict['creativeparaphrase'][premise_df['type'].value_counts()['CreativeParaphrase']] += 1
flute_df['SarcasmSimilarity'] = flute_df.apply(lambda x: label_similar_sarcasm(x,multiple_sarcasm_premise),axis =1)

In [ ]:
def label_similar_hypo(row):
  hypo = row['hypothesis']
  hypothesis_df = grp_by_hypothesis.get_group(hypothesis)
  

In [ ]:
#grp_by_hypothesis = flute_df[flute_df.apply(lambda x : x['Type']!='Sarcasm' and x['Type'] != 'CreativeParaphrase',axis=1)].groupby('Hypothesis')
multiple_hypothesis = []
for hypothesis in grp_by_hypothesis.groups:
  hypothesis_df = grp_by_hypothesis.get_group(hypothesis)
  if len(hypothesis_df)>2:
    print(hypothesis_df)
  
#flute_df['SarcasmSimilarity'] = flute_df.apply(lambda x: label_similar_sarcasm(x,multiple_sarcasm_premise),axis =1)

In [ ]:
flute_test_df = pd.read_json('../Datasets/FLUTE/answer.json', lines=True)
test_premises = flute_test_df['premise'].tolist()

In [ ]:
bool(set(multiple_sarcasm_premise) & set(test_premises))

True

In [ ]:
flute_df[flute_df['premise']=='NO PREMISE FOUND']

,id,hypothesis,premise,idiom,label,explanation,split,type,SarcasmSimilarity
0,1,I was gone for only a few days and my consider...,NO PREMISE FOUND,None,Contradiction,Most people would not consider leaving dirty d...,train,Sarcasm,UNQ
69,70,I am not at all green with envy when I see peo...,NO PREMISE FOUND,None,Contradiction,Owning a washer and dryer is a privilege becau...,train,Sarcasm,UNQ
82,83,I just love that my ex didn't have the decency...,NO PREMISE FOUND,None,Contradiction,Not paying bills like car notes before asking ...,train,Sarcasm,UNQ
108,109,I was so excited to go on this road trip and h...,NO PREMISE FOUND,None,Contradiction,A car breaking down right before a road trip i...,train,Sarcasm,UNQ
111,112,I was really thrilled when I realized I called...,NO PREMISE FOUND,None,Contradiction,Realizing that you called your girlfriend by y...,train,Sarcasm,UNQ
...,...,...,...,...,...,...,...,...,...
7454,7455,I felt like such a great and organized person ...,NO PREMISE FOUND,None,Contradiction,Breaking someone's picture display cannot make...,train,Sarcasm,UNQ
7464,7465,Was looking forward to eating my takeout last ...,NO PREMISE FOUND,None,Contradiction,Half the food being missing from a takeout ord...,train,Sarcasm,UNQ
7473,7474,I really felt the love when my sister was give...,NO PREMISE FOUND,None,Contradiction,A thousand dollars is a lot of money to give a...,train,Sarcasm,UNQ
7504,7505,Got sneezed on by a complete stranger at the b...,NO PREMISE FOUND,None,Contradiction,Getting sneezed on by a stranger is often cons...,train,Sarcasm,UNQ


In [ ]:
cnt_dict

{'sarcasm': {4: 4, 3: 0, 2: 979, 1: 242, 0: 0},
 'creativeparaphrase': {2: 4, 1: 1042, 0: 0}}

In [ ]:
flute_df = flute_df.loc[:,['hypothesis', 'premise', 'type', 'explanation', 'label','SarcasmSimilarity']]
flute_df = flute_df[flute_df['type'] !=  'CreativeParaphrase']
flute_df = flute_df.sort_values(by=['label'])
flute_df = flute_df.drop_duplicates(['hypothesis'], keep='last')
flute_df['Source']= "FLUTE"
flute_df.columns = ['Hypothesis', 'Premise', 'Type', 'Explanation', 'Label','SarcasmSimilarity','Source']


In [ ]:
flute_df['Type'].value_counts()

Sarcasm     2212
Idiom        884
Simile       625
Metaphor     621
Name: Type, dtype: int64

In [ ]:

figqa_ds = load_dataset("nightingal3/fig-qa",split = 'train')
figqa_df = pd.DataFrame.from_dict(figqa_ds)

In [ ]:
figqa_df.head()

,startphrase,ending1,ending2,labels,valid
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0,1
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0,1
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0,1


In [ ]:

figqa_df['Premise'] = figqa_df.apply(lambda x : x['ending1'] if x['labels'] == 0 else x['ending2'],axis=1)
figqa_df['Type'] = 'Metaphor'
figqa_df['Source'] = 'FIGQA'
figqa_df = figqa_df.loc[:,['startphrase','Premise','Type','Source']]

figqa_df.columns = ['Hypothesis','Premise','Type','Source']
figqa_df.head()

,Hypothesis,Premise,Type,Source
0,Her word had the strength of titanium.,Her promises can be believed.,Metaphor,FIGQA
1,Her word had the strength of a wine glass.,Her promises cannot be trusted.,Metaphor,FIGQA
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,Metaphor,FIGQA
3,His kisses have the passion of a couple in a l...,His kiss is unemotional.,Metaphor,FIGQA
4,This winter is as cold as my mother-in-law tow...,It's very cold,Metaphor,FIGQA


In [ ]:
impli_met_df1 = pd.read_csv('../Datasets/IMPLI/met_manual_e.tsv',sep = '\t',header=None)
impli_met_df2 = pd.read_csv('../Datasets/IMPLI/met_replacement_cc_e.tsv',sep = '\t',header=None)
impli_idi_df1 = pd.read_csv('../Datasets/IMPLI/idioms_manual_e.tsv',sep = '\t',header=None)
impli_idi_df2 = pd.read_csv('../Datasets/IMPLI/fig_context_magpie_e.tsv',sep = '\t',header=None)
impli_met_df1  = impli_met_df1.loc[:,]
impli_met_df2  = impli_met_df2.loc[:,]
impli_idi_df1  = impli_idi_df1.loc[:,]
impli_idi_df2 = impli_idi_df2[impli_idi_df2[2]==1.000000]
impli_idi_df2  = impli_idi_df2.loc[:,[0,1]]
impli_met_df1['Type'],impli_met_df2['Type'],impli_idi_df1['Type'],impli_idi_df2['Type'] = 'Metaphor','Metaphor','Idiom','Idiom'
impli_met_df1['Source'],impli_met_df2['Source'],impli_idi_df1['Source'] ,impli_idi_df2['Source']= "IMPLI","IMPLI","IMPLI","IMPLI"
impli_met_df1['Label'],impli_met_df2['Label'],impli_idi_df1['Label'],impli_idi_df2['Label']   = "Entailment","Entailment","Entailment","Entailment"

impli_met_df1.columns,impli_met_df2.columns,impli_idi_df1.columns = [['Hypothesis','Premise','Type','Source','Label'],['Hypothesis','Premise','Type','Source','Label'],['Hypothesis','Premise','Type','Source','Label']]
impli_idi_df2.columns = ['Hypothesis','Premise','Type','Source','Label']

In [ ]:
ab_simile_df = pd.read_csv('../Datasets/AB_COMPARISONS/goldstandard.txt',sep='\t')
ab_simile_df = ab_simile_df.loc[:,['#COMPARISON']]
ab_simile_df.columns = ['Hypothesis']
ab_simile_df['Type'],ab_simile_df['Source'],ab_simile_df['Label'],ab_simile_df['Premise'] = 'Simile','AB','NA','NA'

In [ ]:
ab_simile_df

,Hypothesis,Type,Source,Label,Premise
0,You behave like a toddler on acid.,Simile,AB,NA,NA
1,Your face is like a sweaty mess.,Simile,AB,NA,NA
2,Your manners are like well no manners really.,Simile,AB,NA,NA
3,You look like you're lost.,Simile,AB,NA,NA
4,Your driving style reminds me of a person who ...,Simile,AB,NA,NA
...,...,...,...,...,...
995,Your room is like a cesspit.,Simile,AB,NA,NA
996,Your room is like a padded cell.,Simile,AB,NA,NA
997,Your conversations are like walking through tr...,Simile,AB,NA,NA
998,You are as modern as the queen.,Simile,AB,NA,NA


In [ ]:
simile_df = pd.read_csv('../Datasets/KAGGLE/simile.csv',header=None)
simile_df.columns = ['Hypothesis']
simile_df['Type'],simile_df['Source'],simile_df['Label'],simile_df['Premise'] = 'Simile','Kaggle','NA','NA'

In [ ]:
simile_chatgpt_df = pd.read_csv('../Datasets/CHATGPT/simile_gpt.csv',header=None)
simile_chatgpt_df.columns = ['Hypothesis']
simile_chatgpt_df['Type'],simile_chatgpt_df['Source'],simile_chatgpt_df['Label'],simile_chatgpt_df['Premise'] = 'Simile','CHATGPT','NA','NA'

In [ ]:
simile_df

,Hypothesis,Type,Source,Label,Premise
0,You were as brave as a lion.,Simile,Kaggle,NA,NA
1,They fought like cats and dogs.,Simile,Kaggle,NA,NA
2,He is as funny as a barrel of monkeys.,Simile,Kaggle,NA,NA
3,This house is as clean as a whistle.,Simile,Kaggle,NA,NA
4,He is as strong as an ox.,Simile,Kaggle,NA,NA
...,...,...,...,...,...
204,"It was Francoise, motionless and erect, framed...",Simile,Kaggle,NA,NA
205,The water made a sound like kittens lapping.,Simile,Kaggle,NA,NA
206,Kate inched over her own thoughts like a measu...,Simile,Kaggle,NA,NA
207,"He swung a great scimitar, before which Spania...",Simile,Kaggle,NA,NA


In [ ]:
impli_met_df1

,Hypothesis,Premise,Type,Source,Label
0,he absorbed the knowledge or beliefs of his tr...,he mentally assimilated the knowledge or belie...,Metaphor,IMPLI,Entailment
1,he absorbed the costs for the accident.,he paid for the costs for the accident .,Metaphor,IMPLI,Entailment
2,the sales tax is absorbed into the state incom...,the sales tax is incorporated into the state i...,Metaphor,IMPLI,Entailment
3,the immigrants were quickly absorbed into soci...,the immigrants were quickly assimilated into s...,Metaphor,IMPLI,Entailment
4,her interest in butterflies absorbs her comple...,her interest in butterflies engages her comple...,Metaphor,IMPLI,Entailment
...,...,...,...,...,...
382,"The results are terse and sharply etched , lik...","The results are concise and sharply etched , l...",Metaphor,IMPLI,Entailment
383,"The plane climbs reluctantly , one set of wing...","The plane climbs reluctantly , one set of wing...",Metaphor,IMPLI,Entailment
384,"Torchlights , flaring against the night sky an...","Torchlights , burning against the night sky an...",Metaphor,IMPLI,Entailment
385,Dirty grey clouds tumbled across the sky above...,Dirty grey clouds tumbled across the sky above...,Metaphor,IMPLI,Entailment


In [ ]:

#impli_metaphors = impli_met_df2.sample(n=500)
cmu_metaphors = figqa_df.sample(n=700)
final_df = pd.concat([flute_df,impli_met_df1,cmu_metaphors,impli_met_df2,impli_idi_df1,simile_df,simile_chatgpt_df,ab_simile_df,impli_idi_df2.sample(600)],ignore_index = True)
final_df = final_df.drop_duplicates(['Hypothesis'], keep='last')

In [ ]:
final_df['Type'].value_counts()

Sarcasm     2212
Metaphor    2081
Simile      2019
Idiom       1999
Name: Type, dtype: int64

In [ ]:
final_df

,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source
0,I was gone for only a few days and my consider...,NO PREMISE FOUND,Sarcasm,Most people would not consider leaving dirty d...,Contradiction,UNQ,FLUTE
1,I feel satisfied that the watch I ordered from...,NO PREMISE FOUND,Sarcasm,Receiving a product that is completely differe...,Contradiction,UNQ,FLUTE
2,It makes me feel amazing when I think about th...,NO PREMISE FOUND,Sarcasm,Someone who barely does anything and is not pr...,Contradiction,UNQ,FLUTE
3,I was really looking forward to going biking t...,I was planning to go biking this weekend but i...,Sarcasm,Bike rides are often cancelled due to bad weat...,Contradiction,SIM,FLUTE
4,Delighted with how I had to embarrassingly dan...,I had to dance in front of everyone at the wed...,Sarcasm,Embarrassingly dancing in front of everyone at...,Contradiction,SIM,FLUTE
...,...,...,...,...,...,...,...
8508,You will see where the dramatic storylines are...,You will see where the dramatic storylines are...,Idiom,NaN,Entailment,NaN,IMPLI
8509,"Ten years ago, violence between rival organisa...","Ten years ago, violence between rival organisa...",Idiom,NaN,Entailment,NaN,IMPLI
8510,A case of sink or swim.,A case of fail or succeed in their own effort.,Idiom,NaN,Entailment,NaN,IMPLI
8511,"South African required 201 to win and, though ...","South African required 201 to win and, though ...",Idiom,NaN,Entailment,NaN,IMPLI


In [ ]:
final_df_np = final_df[final_df.Premise == 'NO PREMISE FOUND']
final_df_np = final_df_np.sample(frac=1)
final_df_np[:76].to_csv('../Datasets/Prompting/sarc_npf_AP.csv')
final_df_np[76:152].to_csv('../Datasets/Prompting/sarc_npf_KK.csv')
final_df_np[152:228].to_csv('../Datasets/Prompting/sarc_npf_LS.csv')
final_df_np[228:304].to_csv('../Datasets/Prompting/sarc_npf_PRV.csv')

In [ ]:
df = final_df.sample(50)


In [ ]:
type_counts = final_df['Type'].value_counts()
data_size = len(final_df)
final_df = final_df.sample(frac=1)
prptn_dict = {k:type_counts[k]/data_size for k in type_counts.keys()}
train_count_dict = {k:round(prptn_dict[k]*0.9*data_size) for k in type_counts.keys()}
test_count_dict = {k:round(prptn_dict[k]*0.1*data_size) for k in type_counts.keys()}
sarcasm_sim_df = final_df[final_df.apply(lambda x : x['Type'] =='Sarcasm' and x['SarcasmSimilarity']=='SIM',axis=1)]
sarcasm_unq_df = final_df[final_df.apply(lambda x : x['Type'] =='Sarcasm' and x['SarcasmSimilarity']=='UNQ',axis=1)]
sarc_left_count = int(train_count_dict['Sarcasm']) - len(sarcasm_sim_df)
sarcasm_unq_df_train = sarcasm_unq_df.sample(sarc_left_count)
sarcasm_test_df = sarcasm_unq_df.drop(sarcasm_unq_df_train.index)
sarcasm_train_df = pd.concat([sarcasm_sim_df,sarcasm_unq_df_train])
print(sarcasm_train_df.shape[0],sarcasm_test_df.shape[0],train_count_dict['Sarcasm'],test_count_dict['Sarcasm'])
metaphor_df = final_df[final_df['Type']=='Metaphor']
metaphor_train_df = metaphor_df.sample(train_count_dict['Metaphor'])
metaphor_test_df = metaphor_df.drop(metaphor_train_df.index)
print(metaphor_train_df.shape[0],metaphor_test_df.shape[0],train_count_dict['Metaphor'],test_count_dict['Metaphor'])
simile_df = final_df[final_df['Type']=='Simile']
simile_train_df =simile_df.sample(train_count_dict['Simile'])
simile_test_df = simile_df.drop(simile_train_df.index)
print(simile_train_df.shape[0],simile_test_df.shape[0],train_count_dict['Simile'],test_count_dict['Simile'])
idiom_df = final_df[final_df['Type']=='Idiom']
idiom_train_df =idiom_df.sample(train_count_dict['Idiom'])
idiom_test_df = idiom_df.drop(idiom_train_df.index)
print(idiom_train_df.shape[0],idiom_test_df.shape[0],train_count_dict['Idiom'],test_count_dict['Idiom'])

1991 221 1991 221
1873 208 1873 208
1817 202 1817 202
1799 200 1799 200


In [ ]:
train_df = pd.concat([sarcasm_train_df,metaphor_train_df,simile_train_df,idiom_train_df]).sample(frac=1)
test_df = pd.concat([sarcasm_test_df,metaphor_test_df,simile_test_df,idiom_test_df]).sample(frac=1)

In [ ]:
test_df

,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source
4585,"Let us pass the night together ,",Let us spend the night together,Metaphor,NaN,Entailment,NaN,IMPLI
5244,The man has the build of a third-grader,The man doesn't have any muscles.,Metaphor,NaN,NaN,NaN,FIGQA
620,I am trying to work here so please keep distra...,NO PREMISE FOUND,Sarcasm,Being distracted from work can lead to a lack ...,Contradiction,UNQ,FLUTE
5669,A ânecklineâ is drawn connecting the low p...,A ânecklineâ is drawn connecting the low p...,Metaphor,NaN,Entailment,NaN,IMPLI
4491,"And the Spirit , stooping earthward ,","And the Spirit , lowering itself earthward ,",Metaphor,NaN,Entailment,NaN,IMPLI
...,...,...,...,...,...,...,...
1405,So proud of my son who threw a fit at Walmart ...,NO PREMISE FOUND,Sarcasm,Laying down on the floor and screaming when so...,Contradiction,UNQ,FLUTE
2373,", Sophie rolling her eyes,'Oh, that news will ...","Sophie rolling her eyes,'Oh, that news will be...",Idiom,To go down like a lead balloon means to be an ...,Entailment,UNQ,FLUTE
6435,"Come on , stop playing silly buggers , you 're...","Come on , stop behaving in an annoying way , y...",Idiom,NaN,Entailment,NaN,IMPLI
1494,I am super excited that my mom's old car came ...,NO PREMISE FOUND,Sarcasm,Having a leaky trunk in a car is not a bonus f...,Contradiction,UNQ,FLUTE


In [ ]:
len(test_df[test_df.apply(lambda x: x['Type']=='Simile' and x['Premise'] != 'NA',axis=1)])

59

In [ ]:
test_df_simile_na = test_df[test_df.apply(lambda x: x['Type']=='Simile' and x['Premise'] == 'NA',axis=1)]
test_df_simile_na.sample(41).to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/simile_npf.csv')

In [ ]:
int_df = pd.merge(train_df, test_df, how ='inner', on =['Hypothesis'])
int_df.Type_x.value_counts()

Series([], Name: Type_x, dtype: int64)

In [ ]:
train_sents = train_df['Hypothesis'].tolist()
test_sents = test_df['Hypothesis'].tolist()

In [ ]:
list(set(train_sents).intersection(test_sents))

[]

In [ ]:
bool(set(test_sents) & set(train_sents))

False

In [ ]:
test_df[test_df['Premise']=='NO PREMISE FOUND']

,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source
439,My friend just bought a new Porsche and I feel...,NO PREMISE FOUND,Sarcasm,It would be normal for someone to feel some je...,Contradiction,UNQ,FLUTE
0,I was gone for only a few days and my consider...,NO PREMISE FOUND,Sarcasm,Most people would not consider leaving dirty d...,Contradiction,UNQ,FLUTE
877,I am still really calm from the biggest roller...,NO PREMISE FOUND,Sarcasm,Most people would be terrified after going on ...,Contradiction,UNQ,FLUTE
1881,I'm sure my parents will be so proud of me whe...,NO PREMISE FOUND,Sarcasm,Most people would see robbing money donated to...,Contradiction,UNQ,FLUTE
1508,I feel really calm when my husband leaves his ...,NO PREMISE FOUND,Sarcasm,Dirty clothes on the floor are often an indica...,Contradiction,UNQ,FLUTE
...,...,...,...,...,...,...,...
887,I am so glad that my crush decided to turn me ...,NO PREMISE FOUND,Sarcasm,The person who got turned down probably feels ...,Contradiction,UNQ,FLUTE
1120,I am very pleased that all my cucumber plants ...,NO PREMISE FOUND,Sarcasm,Flooding can kill plants and cucumber plants a...,Contradiction,UNQ,FLUTE
1048,I am really happy about the fact that my brand...,NO PREMISE FOUND,Sarcasm,headlights are an important part of a car and ...,Contradiction,UNQ,FLUTE
251,I woke up this morning to find my mailbox beau...,NO PREMISE FOUND,Sarcasm,A mailbox is not a thing that is meant to be d...,Contradiction,UNQ,FLUTE


In [ ]:
test_df[test_df.apply(lambda x : x['Type'] !='Sarcasm' and x['Label']=='Entailment',axis=1)]

In [ ]:
train_df_cf = train_df.loc[:,['Hypothesis','Type','Source']]
test_df_cf = test_df.loc[:,['Hypothesis','Type','Source']]
#train_df.to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/train.csv')
#train_df_cf.to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Classification/train-0.2.csv')
#test_df.to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/test.csv')
#test_df_cf.to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Classification/test-0.2.csv')


In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/train.csv')

In [ ]:
test_df = pd.read_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/test.csv')
test_df.columns = ['Index', 'Hypothesis', 'Premise', 'Type', 'Explanation', 'Label', 'SarcasmSimilarity', 'Source']

test_df[test_df['Premise'] == 'NA']

,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source


In [ ]:
train_df.columns = ['Index', 'Hypothesis', 'Premise', 'Type', 'Explanation', 'Label', 'SarcasmSimilarity', 'Source']
test_df['Premise'] = test_df['Premise'].fillna(0)
test_df

,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source
0,4585,"Let us pass the night together ,",Let us spend the night together,Metaphor,NaN,Entailment,NaN,IMPLI
1,5244,The man has the build of a third-grader,The man doesn't have any muscles.,Metaphor,NaN,NaN,NaN,FIGQA
2,620,I am trying to work here so please keep distra...,NO PREMISE FOUND,Sarcasm,Being distracted from work can lead to a lack ...,Contradiction,UNQ,FLUTE
3,5669,A ânecklineâ is drawn connecting the low p...,A ânecklineâ is drawn connecting the low p...,Metaphor,NaN,Entailment,NaN,IMPLI
4,4491,"And the Spirit , stooping earthward ,","And the Spirit , lowering itself earthward ,",Metaphor,NaN,Entailment,NaN,IMPLI
...,...,...,...,...,...,...,...,...
826,1405,So proud of my son who threw a fit at Walmart ...,NO PREMISE FOUND,Sarcasm,Laying down on the floor and screaming when so...,Contradiction,UNQ,FLUTE
827,2373,", Sophie rolling her eyes,'Oh, that news will ...","Sophie rolling her eyes,'Oh, that news will be...",Idiom,To go down like a lead balloon means to be an ...,Entailment,UNQ,FLUTE
828,6435,"Come on , stop playing silly buggers , you 're...","Come on , stop behaving in an annoying way , y...",Idiom,NaN,Entailment,NaN,IMPLI
829,1494,I am super excited that my mom's old car came ...,NO PREMISE FOUND,Sarcasm,Having a leaky trunk in a car is not a bonus f...,Contradiction,UNQ,FLUTE


In [ ]:
sarc_npf_AP = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_AP.xlsx')
sarc_npf_KK = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_KK.xlsx')
sarc_npf_LS = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_LS.xlsx')
sarc_npf_PRV = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_PRV.xlsx')
sarc_npf_new = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_new.xlsx')
sarc_npf_new2 = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_new2.xlsx')
sarc_npf_new = sarc_npf_new.drop('Index', axis=1)
sarc_npf_new2 = sarc_npf_new2.drop('Index', axis=1)

simile_npf = pd.read_excel('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/simile_npf.xlsx') 
sarc_npf_df = pd.concat([sarc_npf_AP,sarc_npf_KK,sarc_npf_LS,sarc_npf_PRV,sarc_npf_new])
ann_df = pd.concat([sarc_npf_AP,sarc_npf_KK,sarc_npf_LS,sarc_npf_PRV,simile_npf,sarc_npf_new,sarc_npf_new2])
ann_df.columns = ['Index', 'Hypothesis', 'Premise', 'Type', 'Explanation', 'Label','SarcasmSimilarity', 'Source']
#sarc_npf_df.columns = ['Index', 'Hypothesis', 'Premise', 'Type', 'Explanation', 'Label',
       #'SarcasmSimilarity', 'Source']
ann_df.columns

Index(['Index', 'Hypothesis', 'Premise', 'Type', 'Explanation', 'Label',
       'SarcasmSimilarity', 'Source'],
      dtype='object')

In [ ]:
ann_df.Type.value_counts()

Sarcasm    363
Simile      41
Name: Type, dtype: int64

In [ ]:
ann_df[ann_df['Index']== 1881]

,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source


In [ ]:
def update_premise(row,ann_df):
  Type = row['Type']
  premise = row['Premise']
  id = row['Index']
  Hypothesis = row['Hypothesis']
  if Type == 'Sarcasm' and premise == 'NO PREMISE FOUND':
    if len(ann_df[ann_df.apply(lambda x : x['Hypothesis'] == Hypothesis ,axis =1)]) == 0:
      print(id)
      return premise
    new_premise = ann_df[ann_df.apply(lambda x : x['Hypothesis'] == Hypothesis ,axis =1)]['Premise'].values[0]
    #print(new_premise)
    return new_premise
  elif Type == 'Simile' and premise == 0:
    if len(ann_df[ann_df.apply(lambda x : x['Hypothesis'] == Hypothesis ,axis =1)]) > 0:
      new_premise = ann_df[ann_df.apply(lambda x : x['Hypothesis'] == Hypothesis,axis =1)]['Premise'].values[0]
    #print(new_premise)
      return new_premise
      #return premise
    elif len(ann_df[ann_df.apply(lambda x : x['Index'] == id ,axis =1)]) > 0:
      new_premise = ann_df[ann_df.apply(lambda x : x['Index'] == id,axis =1)]['Premise'].values[0]
    #print(new_premise)
      return new_premise
    #print('yes')
    
  return premise

In [ ]:
sim1 = ann_df[ann_df.apply(lambda x :  x['Type'] == 'Simile' ,axis =1)]['Hypothesis'].tolist()
sim2 = test_df[test_df.apply(lambda x : x['Premise'] == 0 and x['Type'] == 'Simile' ,axis =1)]['Hypothesis'].tolist()
bool(set(sim1) & set(sim2))

False

In [ ]:
#test_df['Premise'] = test_df.apply(lambda x : update_premise(x,ann_df),axis =1)
train_df['Premise'] = train_df.apply(lambda x : update_premise(x,ann_df),axis =1)

In [ ]:
test_npf = test_df[test_df['Premise']=='NO PREMISE FOUND']
train_npf = train_df[train_df['Premise']=='NO PREMISE FOUND']
sarc_npf_new = pd.concat([test_npf,train_npf])
#sarc_npf_new.to_csv('/content/gdrive/MyDrive/685_CP/Datasets/Prompting/Annotated/sarc_npf_new2.csv')
sarc_npf_new

,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source


In [ ]:
train_npf = train_df[train_df['Premise']=='NO PREMISE FOUND']
train_npf

,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source


In [ ]:
test_df[test_df['Premise'] != 'NA']
sarc_test_df = test_df[test_df['Type'] == 'Sarcasm'].sample(100)
simi_test_df = test_df[test_df.apply(lambda x : x['Premise'] != 0 and x['Type'] == 'Simile' ,axis =1)]
meta_test_df = test_df[test_df['Type'] == 'Metaphor'].sample(100)
idio_test_df = test_df[test_df['Type'] == 'Idiom'].sample(100)

In [ ]:
len(simi_test_df)

202

In [ ]:
prompting_test_df = pd.concat([sarc_test_df,simi_test_df,meta_test_df,idio_test_df])

In [ ]:
prompting_test_df.to_csv('../Datasets/Prompting/prompting_test.csv')

In [ ]:
train_df.to_csv('../Datasets/Prompting/train-0.3.csv')